<a href="https://colab.research.google.com/github/Ajay-Kanna-0231/NLP-Project/blob/main/Keyword_Extraction_with_TF_IDF_and_SKlearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd

url = 'https://raw.githubusercontent.com/kavgan/nlp-in-practice/b841c04456370ac45ff5564976fdacc445d3edf0/tf-idf/data/stackoverflow-test.json'
df_idf = pd.read_json(url, lines=True)

# Print the schema and shape of the dataframe
print("Schema:\n\n", df_idf.dtypes)
print("Number of questions, columns=", df_idf.shape)

Schema:

 id                            int64
title                        object
body                         object
accepted_answer_id          float64
answer_count                  int64
comment_count                 int64
creation_date                object
last_activity_date           object
last_edit_date               object
last_editor_display_name     object
last_editor_user_id         float64
owner_display_name           object
owner_user_id               float64
post_type_id                  int64
score                         int64
tags                         object
view_count                    int64
favorite_count              float64
dtype: object
Number of questions, columns= (500, 18)


In [9]:
import re
def pre_process(text):

    # lowercase
    text=text.lower()

    #remove tags
    text=re.sub("</?.*?>"," <> ",text)

    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)

    return text

df_idf['text'] = df_idf['title'] + df_idf['body']
df_idf['text'] = df_idf['text'].apply(lambda x:pre_process(x))

#show the first 'text'
df_idf['text'][2]

'dynamic operations can only be performed in homogenous appdomain i m working with an api that requires lt trust level full legacycasmodel true gt to be set in my web config file in order to work this works without a problem in vs but when i use vs i get an error stating that dynamic operations can only be performed in homogeneous appdomain the project is targeting net framework and is an asp net mvc project i tried changing legacycasmodel to false but then i can t access the object i need not sure how to resolve this issue how can i use legacycasmodel true in vs with dynamic expressions '

In [19]:
import requests
from sklearn.feature_extraction.text import CountVectorizer
import re

def get_stop_words(stop_file_url):
    """Load stop words from a remote URL."""
    response = requests.get(stop_file_url)
    # Check if the request was successful
    if response.status_code == 200:
        stopwords = response.text.splitlines()
        stop_set = set(m.strip() for m in stopwords)
        return list(stop_set)  # Return a list of stopwords
    else:
        print("Failed to retrieve stopwords")
        return []  # Return an empty list if the download failed

# URL to the stopwords file
stopwords_url = "https://raw.githubusercontent.com/kavgan/nlp-in-practice/b841c04456370ac45ff5564976fdacc445d3edf0/tf-idf/resources/stopwords.txt"

# Load a set of stop words
stopwords = get_stop_words(stopwords_url)

# Get the text column from your DataFrame
docs = df_idf['text'].tolist()

# Create a vocabulary of words, ignore words that appear in 85% of documents, eliminate stop words
cv = CountVectorizer(max_df=0.85, stop_words=stopwords)
word_count_vector = cv.fit_transform(docs)


In [20]:
word_count_vector.shape

(500, 8586)

In [22]:
cv=CountVectorizer(max_df=0.85,stop_words=stopwords,max_features=10000)
word_count_vector=cv.fit_transform(docs)
word_count_vector.shape

(500, 8586)

In [24]:
list(cv.vocabulary_.keys())[:10]

['integrate',
 'war',
 'plugin',
 'eclipse',
 'project',
 'set',
 'small',
 'web',
 'jsf',
 'maven']

In [27]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

In [28]:
tfidf_transformer.idf_

array([6.52345892, 6.52345892, 6.52345892, ..., 5.83031174, 6.52345892,
       6.52345892])

In [29]:
# read test docs into a dataframe and concatenate title and body
df_test=pd.read_json("data/stackoverflow-test.json",lines=True)
df_test['text'] = df_test['title'] + df_test['body']
df_test['text'] =df_test['text'].apply(lambda x:pre_process(x))

# get test docs into a list
docs_test=df_test['text'].tolist()
docs_title=df_test['title'].tolist()
docs_body=df_test['body'].tolist()

In [30]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""

    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]

        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]

    return results

In [34]:
# Use this once to get feature names
feature_names = cv.get_feature_names_out()

# Continue with your code
doc = docs_test[0]  # Ensure you have defined 'docs_test' somewhere in your code

# Generate TF-IDF for the given document using your already defined TF-IDF transformer
tf_idf_vector = tfidf_transformer.transform(cv.transform([doc]))

# Sort the TF-IDF vectors by descending order of scores
sorted_items = sort_coo(tf_idf_vector.tocoo())

# Extract only the top n; n here is 10
keywords = extract_topn_from_vector(feature_names, sorted_items, 10)

# Now print the results
print("\n=====Title=====")
print(docs_title[0])  # Ensure 'docs_title' is defined and contains data
print("\n=====Body=====")
print(docs_body[0])  # Ensure 'docs_body' is defined and contains data
print("\n===Keywords===")
for k in keywords:
    print(k, keywords[k])



=====Title=====
Integrate War-Plugin for m2eclipse into Eclipse Project

=====Body=====
<p>I set up a small web project with JSF and Maven. Now I want to deploy on a Tomcat server. Is there a possibility to automate that like a button in Eclipse that automatically deploys the project to Tomcat?</p>

<p>I read about a the <a href="http://maven.apache.org/plugins/maven-war-plugin/" rel="nofollow noreferrer">Maven War Plugin</a> but I couldn't find a tutorial how to integrate that into my process (eclipse/m2eclipse).</p>

<p>Can you link me to help or try to explain it. Thanks.</p>

===Keywords===
eclipse 0.565
tomcat 0.3
integrate 0.3
war 0.281
maven 0.258
project 0.251
plugin 0.236
deploys 0.15
possibility 0.141
jsf 0.141


In [35]:
# put the common code into several methods
def get_keywords(idx):

    #generate tf-idf for the given document
    tf_idf_vector=tfidf_transformer.transform(cv.transform([docs_test[idx]]))

    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,10)

    return keywords

def print_results(idx,keywords):
    # now print the results
    print("\n=====Title=====")
    print(docs_title[idx])
    print("\n=====Body=====")
    print(docs_body[idx])
    print("\n===Keywords===")
    for k in keywords:
        print(k,keywords[k])

In [36]:
idx=120
keywords=get_keywords(idx)
print_results(idx,keywords)


=====Title=====
SQL Import Wizard - Error

=====Body=====
<p>I have a CSV file that I'm trying to import into SQL Management Server Studio.</p>

<p>In Excel, the column giving me trouble looks like this:
<a href="https://i.stack.imgur.com/pm0uS.png" rel="nofollow noreferrer"><img src="https://i.stack.imgur.com/pm0uS.png" alt="enter image description here"></a></p>

<p>Tasks > import data > Flat Source File > select file</p>

<p><a href="https://i.stack.imgur.com/G4b6I.png" rel="nofollow noreferrer"><img src="https://i.stack.imgur.com/G4b6I.png" alt="enter image description here"></a></p>

<p>I set the data type for this column to DT_NUMERIC, adjust the DataScale to 2 in order to get 2 decimal places, but when I click over to Preview, I see that it's clearly not recognizing the numbers appropriately:</p>

<p><a href="https://i.stack.imgur.com/NZhiQ.png" rel="nofollow noreferrer"><img src="https://i.stack.imgur.com/NZhiQ.png" alt="enter image description here"></a></p>

<p>The column ma

In [37]:
#generate tf-idf for all documents in your list. docs_test has 500 documents
tf_idf_vector=tfidf_transformer.transform(cv.transform(docs_test))

results=[]
for i in range(tf_idf_vector.shape[0]):

    # get vector for a single document
    curr_vector=tf_idf_vector[i]

    #sort the tf-idf vector by descending order of scores
    sorted_items=sort_coo(curr_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,10)


    results.append(keywords)

df=pd.DataFrame(zip(docs,results),columns=['doc','keywords'])
df

,doc,keywords
0,integrate war plugin for m eclipse into eclips...,"{'eclipse': 0.565, 'tomcat': 0.3, 'integrate':..."
1,phantomjs node page evaulate seems to hang i h...,"{'sitepage': 0.444, 'evaluate': 0.347, 'conten..."
2,dynamic operations can only be performed in ho...,"{'legacycasmodel': 0.459, 'dynamic': 0.345, 'a..."
3,css with relative url to background image i ha...,"{'bg_damask': 0.473, 'background': 0.387, 'ima..."
4,share canvas image on android hello so i write...,"{'uri': 0.336, 'bitmap': 0.295, 'share': 0.277..."
...,...,...
495,is there any net string format compatible func...,"{'delphi': 0.582, 'compatible': 0.364, 'win': ..."
496,how to handle failed xpath lookup in msxml fro...,"{'node': 0.49, 'oxmldom': 0.364, 'autoit': 0.3..."
497,logo animation don t know where to begin i m t...,"{'logo': 0.53, 'step': 0.382, 'triangle': 0.29..."
498,how to introduce a new variable in coq i was w...,"{'ev_list': 0.544, 'coq': 0.311, 'length': 0.2..."
